# The *don't-touch-my-code* orthogonality problem?

When coding it seems like a good idea to try and mimize having to change the code in a datastructure (class) or a function -- there is always the risk we mess things up.  
Furthermore, we may not even have access to the code we are trying to extend or improve, as it may be, say, hidden in some library, or pehaps be read-only to us.

The following is a bit abstract at the moment, but the examples below hopefully help clarify 
what is actually means.  

It appears that two approaches, a funcional one and objectoriented one, are in some sense orthogonal in the advantages they offer when trying to only "append" code:
1. In the functional style, it is easy to extend the functionality -- just write another function.
On the other hand extending the datastructure causes us to go into all the functions and change them to handle the new structure.
2. In the OO style, adding functionality is the problem, we have to go into all the classes and 
add the functionality. Adding a new datastructure is just a new class, that is easy.

*The raises the question if this orthogonality is fundamental? Or can we somehow overcome it and thereby extend code with **both** functionality and datastructures without having to touch existing code?* **#Don't-touch-my-code**

## An example using summing of numbers
Consider summing some numbers
```
(((1 + 2) + 3) + 4) =  10
```
We may think of the expresion above as a tree
```
              (+)
             /    \
           (+)     (4)
          /   \
        (+)     (3)
       /   \
    (1)     (2)
```
where the nodes are (+) and the leaves (n) are values.  
We would like to define a function `eval` that can evaluate such an expression.  
We will try with the two different approaces, 1. the more functional way and 2. the more obect oriented way.

## Standard object oriented way

We define and interface, `IExpr`, with the functionality `eval` and some classes that implenents it, `Val` and `Add`.

In [ ]:
public interface IExpr
{
    int Eval();
}

public class Val : IExpr
{
    private int _n { get; }
    public Val(int n) { _n = n; }
    public int Eval() => _n;
}

public class Add : IExpr
{
    private IExpr _left, _right;
    public Add(IExpr left, IExpr right) { (_left, _right) = (left, right); }
    public int Eval() => _left.Eval() + _right.Eval();
}

var (one, two, three, four) = (new Val(1), new Val(2), new Val(3), new Val(4));
var expr = new Add(new Add(new Add(one, two), three), four);
Console.WriteLine(expr.Eval());


10


## Extending with new functionality in OO requires changes in old code, i.e. in the classes

Let us try to extend with a nice string representation.
For example
```C#
var expr = new Add(new Val(1), new Val(2));
expr.PrettyString() => "(1 + 2)"
``` 

In [ ]:
public interface IExpr
{
    int Eval();
    string PrettyString();
}

public class Val : IExpr
{
    public int N { get; }
    public Val(int n)
    {
        N = n;
    }

    public int Eval()
    {
        return N;
    }

    public string PrettyString()
    {
        return N.ToString();
    }
}

public class Add : IExpr
{
    public  IExpr Left { get; }
    public  IExpr Right { get; }
    public Add(IExpr left, IExpr right)
    {
        Left = left;
        Right = right;
    }

    public int Eval()
    { 
        return Left.Eval() + Right.Eval();
    }

    public string PrettyString()
    {
        return "(" + Left.PrettyString() + " + " + Right.PrettyString() + ")";
    }
}

var (one, two, three, four) = (new Val(1), new Val(2), new Val(3), new Val(4));
var expr = new Add(new Add(new Add(one, two), three), four);
Console.WriteLine(expr.Eval());
Console.WriteLine(expr.PrettyString());

10


(((1 + 2) + 3) + 4)


## Extending datastructures does not require changes using OO way


Let say we would like to extend our datastructure with multiplication (*), then this is done by just adding a new class.

In [ ]:
public class Mul : IExpr
{
    public IExpr Left {get;}
    public IExpr Right {get;}
    public Mul(IExpr e1, IExpr e2)
    {
        Left = e1;
        Right = e2;
    }

    public int Eval()
    {
        return Left.Eval() * Right.Eval();
    }

    public string PrettyString()
    {
        return "(" + Left.PrettyString() + " * " + Right.PrettyString() + ")";
    }
}

var (one, two) = (new Val(1), new Val(2));
var expr = new Mul(new Add(one, two), two);
Console.Write(expr.Eval());
Console.Write(expr.PrettyString());

6

((1 + 2) * 2)

## Functional way in F#

In [ ]:
type Expr = Val of int | Add of Expr * Expr
// eval: Expr -> int
let rec eval expr =
    match expr with
    | Val n -> n
    | Add (e1, e2) -> eval e1 + eval e2

let one, two, three, four = Val 1, Val 2, Val 3, Val 4
let exprf = Add (Add (Add (one, two), three), four)
printfn "%i" (eval exprf)

10

## Extending with functionality in the functional approach does not require changes to old code

We do not have to touch the old code in this case, and it is pretty elegant, just add a new function

In [ ]:
let rec prettyPrint expr =
    match expr with
    | Val n -> n.ToString()
    | Add (e1, e2) ->  "(" + prettyPrint e1 + " + " + prettyPrint e2 + ")"

printf "%i" (eval exprf)
printf "%s" (prettyPrint exprf)

10

(((1 + 2) + 3) + 4)

## Functional way in C# is also possible

In [ ]:
// Marker interface
public interface IExpr2 { } 

public class Add : IExpr2
{
    public IExpr2 Expr1 { get; }
    public IExpr2 Expr2 { get; }
    public Add(IExpr2 e1, IExpr2 e2) => (Expr1, Expr2) = (e1, e2);
}

public class Val : IExpr2
{
    public int N { get; }
    public Val(int n) => N = n;
}

public static class ExprFuncs
{
    public static int Eval(IExpr2 expr) =>
        expr switch
        {
            Val v => v.N,
            Add a => Eval(a.Expr1) + Eval(a.Expr2),
            _     => throw new ArgumentException()
        };
}

var (one, two, three, four) = (new Val(1), new Val(2), new Val(3), new Val(4));
var expr1 = new Add(new Add(new Add(one, two), three), four);
Console.WriteLine(ExprFuncs.Eval(expr1));


10


## Extending data structures requires us to change in the functional way, i.e add stuff to our functions
 

In [ ]:
type Expr = Val of int | Add of Expr * Expr | Mul of Expr * Expr

let rec eval expr =
    match expr with
    | Val n -> n
    | Add (e1, e2) -> eval e1 + eval e2
    | Mul (e1, e2) -> eval e1 * eval e2

let rec prettyPrint expr =
    match expr with
    | Val n -> n.ToString()
    | Add (e1, e2) -> "(" + prettyPrint e1 + " + " + prettyPrint e2 + ")"
    | Mul (e1, e2) -> "(" + prettyPrint e1 + " * " + prettyPrint e2 + ")" // added

let one, two = Val 1, Val 2
let exprmf = Mul (Add (one, two), two)
printf "%i" (eval exprmf)
printf "%s" (prettyPrint exprmf)


((1 + 2) * 2)

6

## The solution using object algebras

In [ ]:
// abstract factory method
public interface IExprOA<T>
{
    T Val(int n);            // data variant 1
    T Add(T left, T right);  // data variant 2
}

// lets define the eval functionality
public interface IEval
{
    int Eval();
}

// implenting a concrete factory gives a method to work over the data variants (Val and Add)
public class EvalExpr : IExprOA<IEval>
{
    public IEval Val(int n) => new EvalVal(n);
    public IEval Add(IEval left, IEval right) => new EvalAdd(left, right);
}

// implenentation of ieval for val
public class EvalVal : IEval
{
    int _n;
    public EvalVal(int n) => _n = n;
    public int Eval() => _n;
}

// implementation of ieval for add 
public class EvalAdd : IEval
{
    private readonly IEval _l, _r;
    public EvalAdd(IEval left, IEval right) => (_l, _r) = (left, right);
    public int Eval() => _l.Eval() + _r.Eval();
}

public static class Factory<T>
{

    public static T Create(IExprOA<T> oa) 
    { 
        var (one, two, three, four) = (oa.Val(1), oa.Val(2), oa.Val(3), oa.Val(4));
        return oa.Add(oa.Add(oa.Add(one, two), three), four);
    }
}

var evalualbleExpr = Factory<IEval>.Create(new EvalExpr());
evalualbleExpr.Eval()

10

In [ ]:

// lets extend with a pretty printing method
public interface IPrint
{
    string PrettyPrint();   
}

// concrete factory with the  ipritn method
public class PrintExpr : IExprOA<IPrint>
{
    public IPrint Val(int n) => new PrintVal(n);
    public IPrint Add(IPrint left, IPrint right) => new PrintAdd(left, right);
}

public class PrintAdd : IPrint
{
    private readonly IPrint _l, _r;
    public PrintAdd(IPrint left, IPrint right) => (_l, _r) = (left, right);
    public string PrettyPrint() => "(" + _l.PrettyPrint() + " + " +_r.PrettyPrint() + ")";
}

public class PrintVal : IPrint
{
    private readonly int _n;
    public PrintVal(int n) => _n = n;
    public string PrettyPrint() => _n.ToString();
}

public static class Make<T>
{
    public static T MakeExpr(IExprOA<T> a)
    {
        var (one, two, three, four) = (a.Val(1), a.Val(2), a.Val(3), a.Val(4));
        return a.Add(a.Add(a.Add(one, two), three), four);
    }

    public static T MakeMulExpr(IMulOA<T> a)
    {
        var (one, two, three, four) = (a.Val(1), a.Val(2), a.Val(3), a.Val(4));
        return a.Add(a.Mul(a.Add(one, two), three), four);
    }
}

var inp = "1+2+5+4";
var x = Make<IPrint>.MakeExpr(new PrintExpr());
Console.Write(x.PrettyPrint());
var y = Make<IEval>.MakeExpr(new EvalExpr());
Console.WriteLine(y.Eval());

// extending with a data variants
public interface IMulOA<T> : IExprOA<T>
{
    T Mul(T left, T right);
}

// a printer for the new data variant
public class PrintMulExpr : PrintExpr, IMulOA<IPrint>
{
    public IPrint Mul(IPrint left, IPrint right) => new PrintMul(left, right);
}

public class PrintMul : IPrint
{
    private readonly IPrint _l, _r;
    public PrintMul(IPrint left, IPrint right) => (_l, _r) = (left, right);
    public string PrettyPrint() => "(" + _l.PrettyPrint() + " * " +_r.PrettyPrint() + ")";
}

public class EvalMulExpr : EvalExpr, IMulOA<IEval>
{
    public IEval Mul(IEval left, IEval right) => new EvalMul(left, right);
}

public class EvalMul : IEval
{
    private readonly IEval _l, _r;
    public EvalMul(IEval left, IEval right) => (_l, _r) = (left, right);
    public int Eval() => _l.Eval()  * _r.Eval();
}

var x2 = Make<IPrint>.MakeMulExpr(new PrintMulExpr());
Console.Write(x2.PrettyPrint());
var x3 = Make<IEval>.MakeMulExpr(new EvalMulExpr());
Console.Write(x3.Eval());

(((1 + 2) + 3) + 4)

10


(((1 + 2) * 3) + 4)

13